In [52]:
# For following a second Drone
#Must have second drone in sights of Mambo Drone
import numpy as np
import matplotlib.pyplot as plt
import glob
import cv2
import random
import tensorflow as tf
from sklearn.utils import shuffle



In [53]:
import pickle
import zipfile 

In [13]:
training_file = "./traffic-signs-data/train.p"
validation_file= "./traffic-signs-data/valid.p"
testing_file = "./traffic-signs-data/valid.p"

with open(training_file, mode = 'rb') as f:
    train = pickle.load(f)

with open(testing_file, mode = 'rb') as f:
    test = pickle.load(f)
    
with open(validation_file, mode = 'rb') as f:
    valid = pickle.load(f)
    

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']
X_valid, y_valid = valid['features'], valid['features']


In [14]:
import numpy as np

n_train = len(X_train)
n_test = len(X_test)
n_valid = len(X_valid)

#Image shape and dimensions
image_shape = X_train[0].shape
#Number of classes and labels

n_classes = len(np.unique(y_train))

print("Number of training examples = ", n_train)
print("Number of testing examples = ", n_test)
print("Image shape = ", image_shape)
print("Number of classes = ", n_classes)

Number of training examples =  34799
Number of testing examples =  4410
Image shape =  (32, 32, 3)
Number of classes =  43


In [15]:
#Preprocess the Dataset 

#Grayscale 

X_train_gray = np.sum(X_train / 3, axis = 3, keepdims=True)
X_valid_gray = np.sum(X_valid / 3, axis = 3, keepdims=True)
X_test_gray = np.sum(X_test / 3, axis = 3, keepdims= True)
print("X_train_gray", X_train_gray.shape)

X_train_gray (34799, 32, 32, 1)


In [34]:
# Normalize images 
X_train_normalize = ((X_train_gray - 128) / 128)
X_valid_normalize = ((X_valid_gray - 128) / 128)
X_test_normalize = ((X_test_gray - 128) / 128)

print(np.mean(X_train_normalize))
print(np.mean(X_test_normalize))

X_train = X_train_normalize
X_valid = X_valid_normalize
X_test = X_test_normalize

-0.354081335648
-0.347215411128


In [35]:
#Image Augmentation Functions 
import skimage

In [36]:
def sharpen_image(img):
    gb = cv2.GaussianBlur(img, (5,5), 20.0)
    return cv2.addWeighted(img, 2, gb, -1, 0)

In [37]:
def translate_random(img):
    np.random.seed(42)
    rand_x = np.random.randint(-5,5)
    rand_y = np.random.randint(-5,5)
    translation_matrix = np.float32([ [1,0,rand_x], [0,1,rand_y]])
    return cv2.warpAffine(img, translation_matrix, (32, 32))

In [38]:
def add_noise(img):
    noisy_image = skimage.util.random_noise(img, mode='gaussian', seed=None, clip=True) 
    return np.asarray(noisy_image, dtype="uint8")

In [39]:
def augment_all(data):
    for x in data:
        rand_number = random.randint(0, 6)
        if rand_number == 0 or 1:
            x = sharpen_image(x)
        if rand_number == 2:
            x = translate_random(x)
        if rand_number == 3:
            x = add_noise(x)
    return data

In [54]:
from keras.models import Sequential
from keras.layers import Flatten, Dense, Lambda, Cropping2D
from keras.layers import Dropout
from keras.layers import Convolution2D
from keras.layers.pooling import MaxPooling2D
from keras.utils import to_categorical

In [55]:
X_train, y_train = shuffle(X_train, y_train)

In [56]:
#Lanet Architecture
model = Sequential()
#Input: 32, 32, 1  Output: 30, 30, 10
# model.add(Convolution2D(input_shape=(32, 32, 1)))
model.add(Convolution2D(10, 3, 3, subsample=(1, 1), activation='relu' , input_shape=(32, 32, 1)))
model.add(Dropout(0.9))
#Input: 30, 30, 10 Output: 28, 28, 100 
model.add(Convolution2D(100, 3, 3, subsample=(1, 1), activation='relu'))
#Max Pooling Input: 28, 28, 100 Output 14, 14, 100
model.add(MaxPooling2D(2, 2))
#Input: 14, 14, 100 Output: 10, 10, 150
model.add(Convolution2D(150, 5, 5, subsample=(1,1), activation= 'relu'))
model.add(Dropout(0.8))
#Max Pooling Input 10, 10, 150 Output: 5, 5, 150
model.add(MaxPooling2D(2,2))
#Input 5, 5, 150 Output 4, 4, 250
model.add(Convolution2D(250, 2, 2,subsample=(1,1), activation= 'relu'))
model.add(Dropout(0.7))
#Max Pooling Input 4, 4, 250 Output: 2, 2, 250
model.add(MaxPooling2D(2, 2))
#Input 2, 2, 250 Output: 1, 1, 1000
model.add(Convolution2D(1000, 2, 2,subsample=(1,1), activation= 'relu'))
model.add(Flatten())

model.add(Dense(500))
model.add(Dense(300))
model.add(Dense(120))
model.add(Dense(84))
model.add(Dense(n_classes))



model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')
model.fit(X_train, y_train, nb_epoch= 15, validation_split=0.2, batch_size = 120)
model.save('model.h5')

print('saved')

/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(10, (3, 3), activation="relu", input_shape=(32, 32, 1..., strides=(1, 1))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(100, (3, 3), activation="relu", strides=(1, 1))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(150, (5, 5), activation="relu", strides=(1, 1))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(250, (2, 2), activation="relu", strides=(1, 1))`
/home/carnd/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:22: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(1000, (2, 2), activation="relu", strides=(1, 1))`
/home/carnd/ana

Train on 27839 samples, validate on 6960 samples
Epoch 1/15
27839/27839 [==============================] - 262s - loss: 10.3943 - val_loss: 10.7210
Epoch 2/15
27839/27839 [==============================] - 258s - loss: 10.5881 - val_loss: 10.7210
Epoch 3/15
27839/27839 [==============================] - 258s - loss: 10.5886 - val_loss: 10.7210
Epoch 4/15
27839/27839 [==============================] - 258s - loss: 10.7994 - val_loss: 12.8492
Epoch 5/15
27839/27839 [==============================] - 258s - loss: 12.7655 - val_loss: 12.8241
Epoch 6/15
27839/27839 [==============================] - 258s - loss: 12.7659 - val_loss: 12.8241
Epoch 7/15
27839/27839 [==============================] - 258s - loss: 12.7659 - val_loss: 12.8241
Epoch 8/15
27839/27839 [==============================] - 258s - loss: 12.7659 - val_loss: 12.8241
Epoch 9/15
27839/27839 [==============================] - 258s - loss: 12.7659 - val_loss: 12.8241
Epoch 10/15
27839/27839 [==============================] - 2